In [1]:
import numpy as np
import pandas as pd

# SC

In [5]:
dataset = 3
r = {}
mean_r = {}
for dataset in [3]:
    r[dataset] = {}
    mean_r[dataset] = {}
    for cur_fold in range(5):
        r[dataset][cur_fold] = {}
        mean_r[dataset][cur_fold] = {}
        for num_latent in range(1, 51):
            r[dataset][cur_fold][num_latent] = np.load("../code/baseline/sc-non-nested/{}/sc-non-valid-error-{}-{}-{}.npy".format(dataset, dataset, cur_fold, num_latent)).item()
            mean_r[dataset][cur_fold][num_latent] = pd.Series(r[dataset][cur_fold][num_latent]).mean()

In [6]:
best_result = {}
best_param = {}
best_mean = {}
for cluster in [3]:
    best_result[cluster] = {}
    best_mean[cluster] = {}
    best_param[cluster] = {}
    for cur_fold in range(5):
        best_result[cluster][cur_fold] = {} 

In [7]:
for dataset in [3]:
    for cur_fold in range(5):
        best_param[dataset][cur_fold] = pd.Series(mean_r[dataset][cur_fold]).idxmin()
        best_result[dataset][cur_fold] = r[dataset][cur_fold][best_param[dataset][cur_fold]]
        best_mean[dataset][cur_fold] = mean_r[dataset][cur_fold][best_param[dataset][cur_fold]]

In [9]:
np.save("../code/baseline/result/sc-non-disc-param.npy", best_param)

In [10]:
for dataset in [3]:
    for cur_fold in range(5):
        print("python baseline-sc-without-disc-nested.py {} {} {}&".format(dataset, cur_fold, best_param[dataset][cur_fold]))

python baseline-sc-without-disc-nested.py 3 0 2&
python baseline-sc-without-disc-nested.py 3 1 45&
python baseline-sc-without-disc-nested.py 3 2 37&
python baseline-sc-without-disc-nested.py 3 3 33&
python baseline-sc-without-disc-nested.py 3 4 3&


In [11]:
sc_non_best_param = best_param.copy()
sc_non_pred = {}
sc_non_gt = {}
sc_non_error = {}
for cluster in [3]:
    sc_non_pred[cluster] = {}
    sc_non_gt[cluster] = {}
    sc_non_error[cluster] = {}
    for cur_fold in range(5):
        num_latent = sc_non_best_param[cluster][cur_fold]
        sc_non_pred[cluster][cur_fold] = np.load("../code/baseline/sc-non-nested/{}/sc-non-test-pred-{}-{}-{}.npy".format(cluster, cluster, cur_fold, num_latent))
        sc_non_error[cluster][cur_fold] = np.load("../code/baseline/sc-non-nested/{}/sc-non-test-error-{}-{}-{}.npy".format(cluster, cluster, cur_fold, num_latent)).item()

In [12]:
np.save("../code/baseline/result/sc-non-disc-pred.npy", sc_non_pred)
np.save("../code/baseline/result/sc-non-disc-error.npy", sc_non_error)

In [13]:
pd.DataFrame(sc_non_error[3]).mean(axis=1)

dr        103.761267
dw         56.512676
fridge    322.067512
hvac      104.721319
mw         82.017740
dtype: float64

# DSC

In [22]:
r = {}
mean_r = {}
for dataset in [1, 2]:
    r[dataset] = {}
    mean_r[dataset] = {}
    for cur_fold in range(5):
        r[dataset][cur_fold] = {}
        mean_r[dataset][cur_fold] = {}
        for num_latent in range(1, 51):
            r[dataset][cur_fold][num_latent] = {}
            mean_r[dataset][cur_fold][num_latent] = {}
            for iters in range(10, 110, 10):
                r[dataset][cur_fold][num_latent][iters] = np.load("../code/baseline/sc-with-nested/{}/sc-with-valid-error-{}-{}-{}-{}.npy".format(dataset, dataset, cur_fold, num_latent, iters)).item()
                mean_r[dataset][cur_fold][num_latent][iters] = pd.Series(r[dataset][cur_fold][num_latent][iters]).mean()

In [35]:
best_result = {}
best_param = {}
best_mean = {}
for cluster in [1, 2]:
    best_result[cluster] = {}
    best_mean[cluster] = {}
    best_param[cluster] = {}
    for cur_fold in range(5):
        best_result[cluster][cur_fold] = {} 
        best_param[cluster][cur_fold] = {}


In [38]:
for cluster in [1,2]:
    for cur_fold in range(5):
        
        #find the best
#         temp = pd.DataFrame(mean_r[cluster][cur_fold]).idxmin(axis=1)
        best_error = np.Inf
        for latent in range(1, 51):
            for iters in range(10, 110, 10):
                error = mean_r[cluster][cur_fold][latent][iters]
                if error < best_error:
                    best_error = error
                    best_iters = iters
                    best_latent = latent
            best_param[cluster][cur_fold]['num_latent'] = best_latent
            best_param[cluster][cur_fold]['iters'] = best_iters

            best_result[cluster][cur_fold] = r[cluster][cur_fold][best_latent][best_iters]
            best_mean[cluster][cur_fold] = mean_r[cluster][cur_fold][best_latent][best_iters]

In [39]:
np.save("../code/baseline/result/sc-with-disc-param.npy", best_param)

In [42]:
for dataset in [1, 2]:
    for cur_fold in range(5):
        num_latent = best_param[dataset][cur_fold]['num_latent']
        iters = best_param[dataset][cur_fold]['iters']
        print("python baseline-sc-with-disc-nested.py {} {} {} {}&".format(dataset, cur_fold, num_latent, iters))

python baseline-sc-with-disc-nested.py 1 0 1 10&
python baseline-sc-with-disc-nested.py 1 1 1 10&
python baseline-sc-with-disc-nested.py 1 2 1 10&
python baseline-sc-with-disc-nested.py 1 3 1 10&
python baseline-sc-with-disc-nested.py 1 4 1 60&
python baseline-sc-with-disc-nested.py 2 0 7 20&
python baseline-sc-with-disc-nested.py 2 1 7 10&
python baseline-sc-with-disc-nested.py 2 2 6 10&
python baseline-sc-with-disc-nested.py 2 3 1 20&
python baseline-sc-with-disc-nested.py 2 4 3 10&


In [46]:
sc_with_best_param = best_param.copy()
sc_with_pred = {}
sc_with_gt = {}
sc_with_error = {}
for cluster in [1, 2]:
    sc_with_pred[cluster] = {}
    sc_with_gt[cluster] = {}
    sc_with_error[cluster] = {}
    for cur_fold in range(5):
        num_latent = sc_with_best_param[cluster][cur_fold]['num_latent']
        iters = sc_with_best_param[cluster][cur_fold]['iters']
        sc_with_pred[cluster][cur_fold] = np.load("../code/baseline/sc-with-nested/{}/sc-with-test-pred-{}-{}-{}-{}.npy".format(cluster, cluster, cur_fold, num_latent, iters))
        sc_with_error[cluster][cur_fold] = np.load("../code/baseline/sc-with-nested/{}/sc-with-test-error-{}-{}-{}-{}.npy".format(cluster, cluster, cur_fold, num_latent, iters)).item()

In [90]:
np.save("../code/baseline/result/DSC-pred.npy", sc_with_pred)
np.save("../code/baseline/result/DSC-error.npy", sc_with_error)

In [88]:
pd.DataFrame(sc_with_error[2]).mean(axis=1)

dr        92.060578
dw        29.128687
fridge    98.204278
hvac      47.914340
mw        22.252275
dtype: float64

# STF

In [2]:
r = {}
mean_r = {}
for dataset in [1, 2]:
    r[dataset] = {}
    mean_r[dataset] = {}
    for cur_fold in range(5):
        r[dataset][cur_fold] = {}
        mean_r[dataset][cur_fold] = {}
        for num_latent in range(1, 21):
            r[dataset][cur_fold][num_latent] = {}
            mean_r[dataset][cur_fold][num_latent] = {}
            for lr in [0.01, 0.1 ,1 ,2]:
                lr = float(lr)
                r[dataset][cur_fold][num_latent][lr] = {}
                mean_r[dataset][cur_fold][num_latent][lr] = {}
                for iters in range(100, 2600, 400):
                    r[dataset][cur_fold][num_latent][lr][iters] = np.load("../code/baseline/stf/{}/valid/stf-error-{}-{}-{}-{}-{}.npy".format(dataset, dataset, cur_fold, num_latent, lr, iters)).item()
                    mean_r[dataset][cur_fold][num_latent][lr][iters] = pd.Series(r[dataset][cur_fold][num_latent][lr][iters]).mean()

In [3]:
best_result = {}
best_param = {}
best_mean = {}
for cluster in [1, 2]:
    best_result[cluster] = {}
    best_mean[cluster] = {}
    best_param[cluster] = {}
    for cur_fold in range(5):
        best_result[cluster][cur_fold] = {} 
        best_param[cluster][cur_fold] = {}

In [4]:
for dataset in [1, 2]:
    for cur_fold in range(5):
        
        #find the best
        best_error = np.Inf
        for num_latent in range(1, 21):
            for lr in [0.01, 0.1, 1, 2]:
                lr = float(lr)
                for iters in range(100, 2600, 400):
                    error = mean_r[dataset][cur_fold][num_latent][lr][iters]
                    if error < best_error:
                        best_error = error
                        best_lr = lr
                        best_iters = iters
                        best_latent = num_latent
        
        best_param[dataset][cur_fold]['num_latent'] = best_latent
        best_param[dataset][cur_fold]['iters'] = best_iters
        best_param[dataset][cur_fold]['lr'] = best_lr


        
        best_result[dataset][cur_fold] = r[dataset][cur_fold][best_latent][best_lr][best_iters]
        best_mean[dataset][cur_fold] = mean_r[dataset][cur_fold][best_latent][best_lr][best_iters]

In [5]:
np.save("../code/baseline/result/stf-param.npy", best_param)

In [6]:
for dataset in [1, 2]:
    for cur_fold in range(5):
        num_latent = best_param[dataset][cur_fold]['num_latent'] 
        iters = best_param[dataset][cur_fold]['iters']
        lr = best_param[dataset][cur_fold]['lr']
        print("python baseline-stf-nested.py {} {} {} {} {} &".format(dataset, cur_fold, num_latent, lr, iters))

python baseline-stf-nested.py 1 0 18 0.1 2500 &
python baseline-stf-nested.py 1 1 14 2.0 900 &
python baseline-stf-nested.py 1 2 15 2.0 500 &
python baseline-stf-nested.py 1 3 20 0.1 2500 &
python baseline-stf-nested.py 1 4 13 2.0 500 &
python baseline-stf-nested.py 2 0 14 1.0 1700 &
python baseline-stf-nested.py 2 1 15 2.0 2500 &
python baseline-stf-nested.py 2 2 16 2.0 2100 &
python baseline-stf-nested.py 2 3 10 1.0 2500 &
python baseline-stf-nested.py 2 4 12 2.0 2100 &


In [7]:
stf_best_param = best_param.copy()
stf_pred = {}
stf_gt = {}
stf_error = {}
for cluster in [1, 2]:
    stf_pred[cluster] = {}
    stf_gt[cluster] = {}
    stf_error[cluster] = {}
    for cur_fold in range(5):
        num_latent = stf_best_param[cluster][cur_fold]['num_latent']
        iters = stf_best_param[cluster][cur_fold]['iters']
        lr = stf_best_param[cluster][cur_fold]['lr']
        stf_pred[cluster][cur_fold] = np.load("../code/baseline/stf/{}/test/stf-test-pred-{}-{}-{}-{}-{}.npy".format(cluster, cluster, cur_fold, num_latent, lr , iters))
        stf_error[cluster][cur_fold] = np.load("../code/baseline/stf/{}/test/stf-test-error-{}-{}-{}-{}-{}.npy".format(cluster, cluster, cur_fold, num_latent, lr, iters)).item()

In [8]:
np.save("../code/baseline/result/stf-pred-12.npy", stf_pred)

# MTF

In [9]:
r = {}
mean_r = {}
for dataset in [1, 2]:
    r[dataset] = {}
    mean_r[dataset] = {}
    for cur_fold in range(5):
        r[dataset][cur_fold] = {}
        mean_r[dataset][cur_fold] = {}
        for num_latent in range(1, 21):
            r[dataset][cur_fold][num_latent] = {}
            mean_r[dataset][cur_fold][num_latent] = {}
            for lr in [0.01, 0.1 ,1 ,2]:
                lr = float(lr)
                r[dataset][cur_fold][num_latent][lr] = {}
                mean_r[dataset][cur_fold][num_latent][lr] = {}
                for iters in range(100, 2600, 400):
                    r[dataset][cur_fold][num_latent][lr][iters] = np.load("../code/baseline/mtf/{}/valid/mtf-error-{}-{}-{}-{}-{}.npy".format(dataset, dataset, cur_fold, num_latent, lr, iters)).item()
                    mean_r[dataset][cur_fold][num_latent][lr][iters] = pd.Series(r[dataset][cur_fold][num_latent][lr][iters]).mean()

In [10]:
best_result = {}
best_param = {}
best_mean = {}
for cluster in [1, 2]:
    best_result[cluster] = {}
    best_mean[cluster] = {}
    best_param[cluster] = {}
    for cur_fold in range(5):
        best_result[cluster][cur_fold] = {} 
        best_param[cluster][cur_fold] = {}

In [11]:
for dataset in [1, 2]:
    for cur_fold in range(5):
        
        #find the best
        best_error = np.Inf
        for num_latent in range(1, 21):
            for lr in [0.01, 0.1, 1, 2]:
                lr = float(lr)
                for iters in range(100, 2600, 400):
                    error = mean_r[dataset][cur_fold][num_latent][lr][iters]
                    if error < best_error:
                        best_error = error
                        best_lr = lr
                        best_iters = iters
                        best_latent = num_latent
        
        best_param[dataset][cur_fold]['num_latent'] = best_latent
        best_param[dataset][cur_fold]['iters'] = best_iters
        best_param[dataset][cur_fold]['lr'] = best_lr


        
        best_result[dataset][cur_fold] = r[dataset][cur_fold][best_latent][best_lr][best_iters]
        best_mean[dataset][cur_fold] = mean_r[dataset][cur_fold][best_latent][best_lr][best_iters]

In [12]:
np.save("../code/baseline/result/mtf-param.npy", best_param)

In [13]:
for dataset in [1, 2]:
    for cur_fold in range(5):
        num_latent = best_param[dataset][cur_fold]['num_latent'] 
        iters = best_param[dataset][cur_fold]['iters']
        lr = best_param[dataset][cur_fold]['lr']
        print("python baseline-mtf-nested.py {} {} {} {} {} &".format(dataset, cur_fold, num_latent, lr, iters))

python baseline-mtf-nested.py 1 0 20 0.1 1700 &
python baseline-mtf-nested.py 1 1 15 1.0 900 &
python baseline-mtf-nested.py 1 2 19 2.0 2500 &
python baseline-mtf-nested.py 1 3 7 2.0 100 &
python baseline-mtf-nested.py 1 4 13 2.0 2100 &
python baseline-mtf-nested.py 2 0 17 1.0 900 &
python baseline-mtf-nested.py 2 1 13 2.0 2500 &
python baseline-mtf-nested.py 2 2 8 2.0 2100 &
python baseline-mtf-nested.py 2 3 11 2.0 2500 &
python baseline-mtf-nested.py 2 4 16 1.0 1300 &


In [14]:
mtf_best_param = best_param.copy()
mtf_pred = {}
mtf_gt = {}
mtf_error = {}
for cluster in [1, 2]:
    mtf_pred[cluster] = {}
    mtf_gt[cluster] = {}
    mtf_error[cluster] = {}
    for cur_fold in range(5):
        num_latent = mtf_best_param[cluster][cur_fold]['num_latent']
        iters = mtf_best_param[cluster][cur_fold]['iters']
        lr = mtf_best_param[cluster][cur_fold]['lr']
        mtf_pred[cluster][cur_fold] = np.load("../code/baseline/mtf/{}/test/mtf-test-pred-{}-{}-{}-{}-{}.npy".format(cluster, cluster, cur_fold, num_latent, lr , iters))
        mtf_error[cluster][cur_fold] = np.load("../code/baseline/mtf/{}/test/mtf-test-error-{}-{}-{}-{}-{}.npy".format(cluster, cluster, cur_fold, num_latent, lr, iters)).item()

In [15]:
np.save("../code/baseline/result/mtf-pred-12.npy", mtf_pred)